Here are three minimal exercises that show:
1. Functional Programming works
2. However, adding some extra functionality can make Functional Programming painful or prone to failure.
3. Introduction to a very simple OOP class to handle the same expansion.
4. Successfull handling of the problem and its expansion with OOP

### Exercise 1: The counter

#### 1.1 Create the counter with Fucntional Programming

In [1]:
def create_counter(start):
    return start

def increase(x):
    return x + 1

counter = create_counter(0)
counter = increase(counter)
counter = increase(counter)
print(counter)  # Correct Answer: 2


2


The result is correct and expected. Now lets call the increament multiple times. And during calling the increament, suppose that someone **forgets reassignment**

#### 1.2 A likely mistake to happen with Functional Programming

In [2]:
def create_counter(start):
    return start

def increament(x):
    return x + 1

counter = create_counter(0)

increament(counter)          # BUG: result ignored
counter = increament(counter)

print(counter)        # prints 1, but you might expect 2


1


What failed?
- FP didn’t fail mathematically. It failed humanly: silent misuse is easy.

#### 1.3 Lets reintroduce the same counter with OOP.

In [3]:
class Counter:
    def __init__(self, start):
        self.value = start

    def increament(self):
        self.value += 1


#### 1.4 Expansion succeeds with OOP (bug becomes hard to make)

In [4]:
counter = Counter(0)

counter.increament()
counter.increament()

print(counter.value)  # Correct answer: 2

2


- Why OOP “wins” here: there’s no “forgot to reassign” failure mode.

### Exercise 2: The Shopping Cart

#### 2.1 Lets create a simple shopping cart using Functional programming only

In [5]:
def create_cart():
    return {"items": [], "total": 0.0}

def add_item(cart, name, price):
    return {
        "items": cart["items"] + [(name, price)],
        "total": cart["total"] + price
    }

cart = create_cart()
cart = add_item(cart, "apple", 1.5)
cart = add_item(cart, "banana", 2.0)

print(cart["total"])  # Correct Answer: 3.5


3.5


The answer is correct. Let's expand the functionality of shopping cart by adding:
1. Discount coupon rule
2. Logging every change

#### 2.2 Minimal expansion where FP becomes painful

In [6]:
def logger(msg):
    print(msg)

def add_item(cart, name, price, discount_coupon_percent, logger):
    discounted_price = price * (1 - discount_coupon_percent)
    logger(f"Added {name} for {discounted_price}")

    return {
        "items": cart["items"] + [(name, discounted_price)],
        "total": cart["total"] + discounted_price
    }

cart = {"items": [], "total": 0.0}
cart = add_item(cart, "apple", 1.5, discount_coupon_percent=0.10, logger=logger)
cart = add_item(cart, "banana", 2.0, discount_coupon_percent=0.10, logger=logger)
print(cart["total"])


Added apple for 1.35
Added banana for 1.8
3.1500000000000004


What went wrong?
The Fucntional Programming did not break, but there are other issues.
- The function signature grew significantly.
- Every caller must call the same congif repeatedly.
- Easier to forget to pass discount coupon or logger consistently.

Thats the real-life failure: Complexity spreads outward

#### 2.3 Lets try rebuilding it with the help of a class.

In [7]:
class Cart:
    def __init__(self, discount_coupon_percent, logger):
        self.items = []
        self.total = 0.0
        self.discount_coupon_percent = discount_coupon_percent
        self.logger = logger

    def add_item(self, name, price):
        discounted = price * (1 - self.discount_coupon_percent)
        self.items.append((name, discounted))
        self.total += discounted
        self.logger(f"Added {name} for {discounted}")


#### 2.4 Expansion with OOP, cleaner and elegant.

In [8]:
def logger(msg):
    print(msg)

cart = Cart(discount_coupon_percent=0.10, logger=logger)
cart.add_item("apple", 1.5)
cart.add_item("banana", 2.0)

print(cart.total)

Added apple for 1.35
Added banana for 1.8
3.1500000000000004


Why OOP “wins” here:

- config is stored once (coupon_percent, logger)
- callers don’t thread state/config everywhere
- rules live next to the data they affect

### Exercise 3: Rate-limited API calls
#### 3.1 Using functional programming to call an API and count calls 

In [9]:
def call_api(state):
    # state is just a dict
    state = state.copy()
    state["calls"] += 1
    return state, f"response #{state['calls']}"

state = {"calls": 0}

state, r1 = call_api(state)
print(f"State: {state}, Response: {r1}")

state, r2 = call_api(state)
print(f"State: {state}, Response: {r2}")

State: {'calls': 1}, Response: response #1
State: {'calls': 2}, Response: response #2


#### 3.2 Minimal expansion where functional programming becomes annoying as well as error-prone.

Lets consider a new requirement: “Add rate limit + last call time + logging”. 

Now `state` and `config` are everywhere: 

In [10]:
import time

def call_api(state, max_calls, logger):
    state = state.copy()

    if state["calls"] >= max_calls:
        logger("Rate limit hit")
        return state, None

    state["calls"] += 1
    state["last_call"] = time.time()
    logger(f"Called API ({state['calls']}/{max_calls})")
    return state, f"response #{state['calls']}"

def logger(msg):
    print(msg)

state = {"calls": 0, "last_call": None}
state, r1 = call_api(state, max_calls=2, logger=logger)
print(f"State: {state}, Response: {r1}")

state, r2 = call_api(state, max_calls=2, logger=logger)
print(f"State: {state}, Response: {r2}")

state, r3 = call_api(state, max_calls=2, logger=logger)  # now blocked
print(f"State: {state}, Response: {r3}")

Called API (1/2)
State: {'calls': 1, 'last_call': 1768801335.1480985}, Response: response #1
Called API (2/2)
State: {'calls': 2, 'last_call': 1768801335.148499}, Response: response #2
Rate limit hit
State: {'calls': 2, 'last_call': 1768801335.148499}, Response: None


- Again: FP is “possible”, but config/state threading spreads.

#### 3.3 Minimal Class

In [11]:
import time

class APIClient:
    def __init__(self, max_calls, logger):
        self.max_calls = max_calls
        self.logger = logger
        self.calls = 0
        self.last_call = None

    def call(self):
        if self.calls > self.max_calls:
            self.logger("Rate limit hit")
            return None

        self.calls += 1
        self.last_call = time.time()
        self.logger(f"Called API ({self.calls}/{self.max_calls})")
        return f"response #{self.calls}"

#### 3.4 Successful expansion using OOP

In [12]:
def logger(msg):
    print(msg)

client = APIClient(max_calls=2, logger=logger)
print(client.call())
print(client.call())
print(client.call())  # blocked


Called API (1/2)
response #1
Called API (2/2)
response #2
Called API (3/2)
response #3


- Why OOP “wins” here:
    - client has a lifecycle and internal state
    - rate limiting is enforced centrally
    - code using it stays simple

--- 